# Exploring the Data
This is using NY data. Looks like inside aribnb data is same format so it can be used for other cities. This notebook is dealing with selecting columns to use.


In [68]:
import os
import io
import re
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [69]:
#read data, specify index so it's easier to join and search using loc
path = '../../data/new-york-city-airbnb-open-data/'
listings_csv = os.path.join(path,'listings.csv')

listings = pd.read_csv(listings_csv, index_col = 'id')

In [70]:
#shape
total_row = listings.shape[0]
listings.shape

(50796, 105)

In [71]:
#to get all columns, set option
pd.set_option('display.max_columns', 107)
#to get text with no truncation
pd.set_option('display.max_colwidth', -1)

In [72]:
#check if columns have lots of null (50% or more)
def less_than_50_percent(colname):
    isnull_count = listings[colname].isna().sum()
    if isnull_count/total_row > .5:
        return True


In [73]:
columns = list(listings)
remove_columns = []
for column in columns:
    remove_column_y_n = less_than_50_percent(column)
    if remove_column_y_n:
        remove_columns.append(column)

print(remove_columns)

['notes', 'thumbnail_url', 'medium_url', 'xl_picture_url', 'square_feet', 'weekly_price', 'monthly_price', 'license', 'jurisdiction_names']


In [74]:
listings.drop(remove_columns, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 96)

In [78]:
#check if all records have same value
def all_same_value(colname):
    count_unique = len(listings[colname].unique())
    if count_unique == 1:
        return True

In [79]:
columns = list(listings)
remove_columns = []
for column in columns:
    remove_column_y_n = all_same_value(column)
    if remove_column_y_n:
        remove_columns.append(column)

In [81]:
listings.drop(remove_columns, inplace=True, axis=1)
#chek if columns are removed
listings.shape

(50796, 89)